<a href="https://colab.research.google.com/github/Jeanne0410/defia/blob/rfm-jeanne/test_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import dask
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
# Import libraries
from tqdm import tqdm
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

train_dataframes = []
for i in tqdm(range(1, 2)):
    train_dataframes.append(pd.read_csv(f'train_data_part_{i}.csv'))
train_data = pd.concat(train_dataframes, ignore_index=True)

# free up memory by deleting the dataframes we no longer need
del train_dataframes

# This code reads the data from a CSV file named "products_data.csv" into a pandas DataFrame
products_data = pd.read_csv('products_data.csv')

# This code reads the data from a CSV file named "test_data.csv" into a pandas DataFrame
test_data = pd.read_csv('test_data.csv')

100%|██████████| 1/1 [00:01<00:00,  1.07s/it]
<ipython-input-2-47f0d5653ac2>:15: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  products_data = pd.read_csv('products_data.csv')


In [3]:
columns_to_keep = ['date', 'transaction_id', 'customer_id', 'product_id']
train_data = train_data[columns_to_keep]

In [4]:
import pandas as pd

# Assurer que la colonne 'date' est en format datetime
train_data['date'] = pd.to_datetime(train_data['date'])

# Extraire les informations temporelles
train_data['month'] = train_data['date'].dt.month        # Mois de l'année
#train_data['day_of_week'] = train_data['date'].dt.weekday    # jour de la semaine (0 = lundi, 6 = dimanche)
#l'information sur les jours de la semaine aurai pu etre interéssante mais pour les données test nous ne savons pas sur quel jour ce passe les achats

# Afficher un aperçu des données
print(train_data.head())

# Suppression de la colonne date si elle n'est plus nécessaire pour éviter la multicolinéarité
train_data = train_data.drop(columns=['date'])

        date       transaction_id   customer_id     product_id  month
0 2023-11-15  Transaction_1730125  Household_39   Product_5362     11
1 2022-07-20  Transaction_1560535  Household_39  Product_67174      7
2 2022-07-20  Transaction_1560535  Household_39  Product_82254      7
3 2023-11-15  Transaction_1730125  Household_39   Product_3895     11
4 2022-07-20  Transaction_1560535  Household_39  Product_34014      7


In [5]:
# Filtrer pour les mois de janvier et fevrier
train_data_filtered = train_data[train_data['month'].isin([1, 2])]

# Vérifier les données filtrées
print(train_data_filtered['month'].value_counts())

month
1    55411
2    49267
Name: count, dtype: int64


In [6]:
# Identifier les produits les plus fréquents
top_products = train_data_filtered['product_id'].value_counts().nlargest(1000).index
print(top_products)
#train_data_filtered['product_id'] = train_data_filtered['product_id'].apply(lambda x: x if x in top_products else 'autre')
# Convertir top_products en un ensemble pour des recherches plus rapides
top_products_set = set(top_products)

# Filtrer les lignes où product_id est dans top_products
train_data_filtered = train_data_filtered[train_data_filtered['product_id'].isin(top_products_set)]

# Réinitialiser l'index pour garder un DataFrame propre
train_data_filtered = train_data_filtered.reset_index(drop=True)
print(train_data_filtered.shape)
print(train_data.shape)

Index(['Product_23971', 'Product_42748', 'Product_57942', 'Product_2793',
       'Product_40951', 'Product_51497', 'Product_73736', 'Product_9790',
       'Product_63301', 'Product_36019',
       ...
       'Product_5153', 'Product_24381', 'Product_54212', 'Product_27854',
       'Product_23166', 'Product_11869', 'Product_41604', 'Product_38396',
       'Product_74744', 'Product_44282'],
      dtype='object', name='product_id', length=1000)
(41896, 4)
(643828, 4)


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical

# Séparer X et y
X = train_data_filtered.drop(columns=["product_id"])  # Toutes les colonnes sauf product_id
y = train_data_filtered["product_id"]  # La cible (product_id)

# Encoder la variable cible
product_encoder = LabelEncoder()
y_encoded = product_encoder.fit_transform(y)
y_encoded = to_categorical(y_encoded)

# Encoder les colonnes catégoriques dans X
categorical_columns = X.select_dtypes(include=["object"]).columns

# Ordinal encoding pour les colonnes catégoriques
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
X[categorical_columns] = encoder.fit_transform(X[categorical_columns])

# Standardisation des colonnes numériques
numerical_columns = X.select_dtypes(exclude=["object"]).columns
scaler = StandardScaler()
X[numerical_columns] = scaler.fit_transform(X[numerical_columns])

# Conversion en matrice NumPy
X = X.to_numpy()

# Reshape pour Conv1D
X = X.reshape((X.shape[0], X.shape[1], 1))

# Diviser en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# 2. Construire le modèle CNN
model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1), padding='same'), # Added padding='same'
    MaxPooling1D(pool_size=2),
    Dropout(0.25),
    #Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'), # Added padding='same', Reduced filters and kernel_size
    #MaxPooling1D(pool_size=2),
    #Dropout(0.25),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(y_train.shape[1], activation='softmax')  # Nombre de classes (produits)
])

# 3. Compiler le modèle
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [1]:
# 4. Entraîner le modèle
model.fit(X_train, y_train, epochs=10, batch_size=256, validation_data=(X_test, y_test))


NameError: name 'model' is not defined

In [11]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, OrdinalEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 1. Préparation des données
X = train_data_filtered.drop(columns=["product_id"])
y = train_data_filtered["product_id"]

# Encodage des labels
product_encoder = LabelEncoder()
y_encoded = product_encoder.fit_transform(y)
y_encoded = to_categorical(y_encoded)

# Encodage ordinal des colonnes catégoriques
categorical_columns = X.select_dtypes(include=["object"]).columns
encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
X[categorical_columns] = encoder.fit_transform(X[categorical_columns])

# Standardisation des colonnes numériques
numerical_columns = X.select_dtypes(exclude=["object"]).columns
scaler = StandardScaler()
X[numerical_columns] = scaler.fit_transform(X[numerical_columns])

# Conversion en matrice NumPy
X = X.to_numpy()

# Padding pour uniformiser les dimensions (si nécessaire)
X = pad_sequences(X, padding="post", value=0)

# Reshape pour Conv1D
X = X.reshape((X.shape[0], X.shape[1], 1))

# Division en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# 2. Définition du modèle CNN
model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1), padding='same'),
    MaxPooling1D(pool_size=2),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(y_train.shape[1], activation='softmax')
])

# 3. Compilation
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# 4. Ajout d'un callback pour stopper l'entraînement en cas de sur-apprentissage
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# 5. Entraînement du modèle avec des batchs
batch_size = 512  # Réduit la charge mémoire
epochs = 10

model.fit(X_train, y_train, validation_split=0.2, batch_size=batch_size, epochs=epochs, callbacks=[early_stopping])

# 6. Évaluation sur le jeu de test
test_loss, test_accuracy = model.evaluate(X_test, y_test, batch_size=batch_size)
print(f"Test Accuracy: {test_accuracy:.2f}")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_4 (Conv1D)                    │ (None, 3, 64)               │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_4 (MaxPooling1D)       │ (None, 1, 64)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 1, 64)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1000)                │         129,000 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 137,576 (537.41 KB)

 Trainable params: 137,576 (537.41 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 8s 82ms/step - accuracy: 0.0048 - loss: 6.8893 - val_accuracy: 0.0079 - val_loss: 6.7727
Epoch 2/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.0089 - loss: 6.7329 - val_accuracy: 0.0076 - val_loss: 6.7481
Epoch 3/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0085 - loss: 6.7082 - val_accuracy: 0.0084 - val_loss: 6.7436
Epoch 4/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0074 - loss: 6.6986 - val_accuracy: 0.0082 - val_loss: 6.7410
Epoch 5/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0086 - loss: 6.6926 - val_accuracy: 0.0089 - val_loss: 6.7409
Epoch 6/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0090 - loss: 6.6826 - val_accuracy: 0.0082 - val_loss: 6.7430
Epoch 7/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0085 - loss: 6.6714 - val_accuracy: 0.0092 - val_loss: 6.7434
Epoch 8/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0093 - loss: 6.6765 - val_accuracy: 0.0087 - val_loss

In [12]:
def predict_product(model, X_test, product_encoder, top_n=10):
    """
    Génère les produits recommandés pour chaque instance dans X_test.

    Args:
        model: Le modèle entraîné (CNN).
        X_test: Données d'entrée pour la prédiction.
        product_encoder: L'objet LabelEncoder utilisé pour encoder les produits.
        top_n: Le nombre de produits recommandés (par défaut : 10).

    Returns:
        Une liste de listes où chaque sous-liste contient les identifiants des produits recommandés pour chaque exemple de test.
    """
    # Prédire les probabilités pour les données de test
    predictions = model.predict(X_test, batch_size=512)  # Utilisation de batchs pour limiter la mémoire

    # Obtenir les indices des produits les plus probables
    top_products_indices = np.argsort(predictions, axis=1)[:, -top_n:][:, ::-1]  # Indices triés des top N prédictions

    # Décoder les indices pour retourner les identifiants des produits
    top_products = [
        product_encoder.inverse_transform(top_products_indices[i])
        for i in range(len(top_products_indices))
    ]

    return top_products


In [13]:
# Appeler la fonction pour prédire les produits
predicted_products = predict_product(model, X_test, product_encoder, top_n=10)

# Exemple : Afficher les 5 premières prédictions
print("Exemples de prédictions :")
for i, products in enumerate(predicted_products[:5]):
    print(f"Instance {i+1}: {products}")


17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step
Exemples de prédictions :
Instance 1: ['Product_23971' 'Product_42748' 'Product_57942' 'Product_40951'
 'Product_2793' 'Product_51497' 'Product_9790' 'Product_73736'
 'Product_63301' 'Product_1370']
Instance 2: ['Product_42748' 'Product_57942' 'Product_23971' 'Product_40951'
 'Product_2793' 'Product_51497' 'Product_9790' 'Product_63301'
 'Product_73736' 'Product_22371']
Instance 3: ['Product_23971' 'Product_42748' 'Product_57942' 'Product_40951'
 'Product_2793' 'Product_51497' 'Product_9790' 'Product_73736'
 'Product_63301' 'Product_1370']
Instance 4: ['Product_23971' 'Product_42748' 'Product_57942' 'Product_40951'
 'Product_2793' 'Product_51497' 'Product_9790' 'Product_73736'
 'Product_1370' 'Product_63301']
Instance 5: ['Product_23971' 'Product_42748' 'Product_57942' 'Product_40951'
 'Product_2793' 'Product_51497' 'Product_9790' 'Product_73736'
 'Product_63301' 'Product_1370']


In [15]:
def hitrate_at_k(test_data, predicted_products, product_encoder, k=10):
    """
    Calcule le Hitrate@K.

    Args:
        test_data: Le jeu de données de test contenant les produits réellement achetés.
        predicted_products: Liste des produits recommandés par le modèle (liste de listes).
        product_encoder: L'encodeur utilisé pour encoder les produits.
        k: Le nombre de recommandations à considérer (par défaut : 10).

    Returns:
        Le score Hitrate@K.
    """
    # Assurez-vous que test_data est aligné sur les prédictions
    if len(test_data) != len(predicted_products):
        raise ValueError("Le nombre d'exemples dans test_data et predicted_products doit être identique.")

    # Récupérer les vrais produits dans leur forme encodée
    true_products = test_data["product_id"].map(lambda x: product_encoder.transform([x])[0])

    hits = 0

    # Parcourir les produits réels et les recommandations
    for i, true_product in enumerate(true_products):
        if true_product in predicted_products[i][:k]:  # Vérifie si le produit réel est dans les top-k recommandations
            hits += 1

    # Calcul du Hitrate@K
    return hits / len(test_data)

# Calculer Hitrate@10
hitrate = hitrate_at_k(test_data, predicted_products, product_encoder, k=10)

print(f"Hitrate@10 : {hitrate:.2f}")


InvalidArgumentError: Graph execution error:

Detected at node data defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/usr/local/lib/python3.10/dist-packages